In [1]:
# Setup block

import csv

import pyrankvote

# Must be formatted like the question titles in the Engage election
single_seat_positions = [
    "President",
    "Vice President",
    "Secretary",
    "Treasurer",
    "Judicial Chair",
    "Social Media and Web Chair",
    "Merch Chair",
    "Sustainability Chair",
    "Inclusivity Chair",
    "Art Chair",
    "Mural Chair",
    "Social Spaces Chair",
    "Music Room Chair",
    "Makerspace Chair",
    "Gym and Dance Room Chair",
]

multi_seat_positions = {
    "Room Assignment Chair (RAC) - electing 3": 3,
    "Social Chair - electing 3": 3,
    "CPW/REX (Rush) Chair - electing 3": 3,
    "i3 Chair - electing 2": 2,
    "Alumni Relations Chair - electing 2": 2,
}

votes = csv.DictReader(open("survey_answers_mit_1216175055.csv", "r"))

In [2]:
# Figure out how many candidates are running for each position
# by using the CSV field names

num_candidates_by_position = {}
for position in single_seat_positions + list(multi_seat_positions):
    num_candidates_by_position[position] = sum(
        f.startswith(position) for f in votes.fieldnames
    )

num_candidates_by_position

{'President': 1,
 'Vice President': 1,
 'Secretary': 3,
 'Treasurer': 1,
 'Judicial Chair': 4,
 'Social Media and Web Chair': 1,
 'Merch Chair': 2,
 'Sustainability Chair': 1,
 'Inclusivity Chair': 1,
 'Art Chair': 1,
 'Mural Chair': 1,
 'Social Spaces Chair': 3,
 'Music Room Chair': 3,
 'Makerspace Chair': 4,
 'Gym and Dance Room Chair': 4,
 'Room Assignment Chair (RAC) - electing 3': 7,
 'Social Chair - electing 3': 4,
 'CPW/REX (Rush) Chair - electing 3': 5,
 'i3 Chair - electing 2': 4,
 'Alumni Relations Chair - electing 2': 2}

First collect names of candidates by scanning all rows, and convert into the candidate class that pyrankvote uses.

- Columns of ranked votes are in the format `position - rank`, so we look through everyone's votes to collect a set of all the candidates.
- This is needed so we can set up the elections properly, in the next block.
- We also collect all the ballots at the same time, by just looking at the order which the candidates are listed for each voter.

In [3]:
candidates_for_positions = {}
ballots_for_positions = {}

for voter in votes:
    for position, num in num_candidates_by_position.items():
        ballot = []

        for rank in range(1, num + 1):
            candidate_name = voter[f"{position} - {rank}"]
            if candidate_name:
                candidate = pyrankvote.Candidate(candidate_name)
                candidates_for_positions.setdefault(position, set()).add(candidate)
                ballot.append(candidate)

        ballots_for_positions.setdefault(position, list()).append(
            pyrankvote.Ballot(ballot)
        )

In [4]:
candidates_for_positions

{'President': {<Candidate('Kyna McGill')>},
 'Vice President': {<Candidate('Rishika Bansal')>},
 'Secretary': {<Candidate('Ekanem Okeke')>,
  <Candidate('Mingpei 'Ming' Li')>,
  <Candidate('Tasnim Zulfiqar')>},
 'Treasurer': {<Candidate('Mackenzie Bivin')>},
 'Judicial Chair': {<Candidate('KC')>,
  <Candidate('Liam Coy')>,
  <Candidate('Richard Zhu')>,
  <Candidate('Siddhu Pachipala')>},
 'Social Media and Web Chair': {<Candidate('Fernando Valenzuela')>},
 'Merch Chair': {<Candidate('Haydn Long ')>, <Candidate('Sophie Thompson')>},
 'Sustainability Chair': {<Candidate('Haylea Brock')>},
 'Inclusivity Chair': {<Candidate('Emily Wong')>},
 'Art Chair': {<Candidate('Mingpei 'Ming' Li')>},
 'Mural Chair': {<Candidate('Luka Srsic')>},
 'Social Spaces Chair': {<Candidate('Caitlin Ogoe ')>,
  <Candidate('Jonathan Liang')>,
  <Candidate('Josh Simon')>},
 'Music Room Chair': {<Candidate('Alexander Shi Jin')>,
  <Candidate('Jay Mancera')>,
  <Candidate('Kylee Cogdill')>},
 'Makerspace Chair': {<

In [ ]:
ballots_for_positions

In [ ]:
# Now, run the elections!

elected_gov = {}

# Instant Runoff elections for single chair elections

for position in single_seat_positions:
    election = pyrankvote.instant_runoff_voting(
        candidates_for_positions[position], ballots_for_positions[position]
    )
    winner = election.get_winners()
    print(f"=========== {position.upper()} " + "="*(25-len(position)))
    print(election)
    elected_gov[position] = winner

In [ ]:
# Single Transferrable Vote elections for multiple chair elections

for position, seats in multi_seat_positions.items():
    election = pyrankvote.single_transferable_vote(
        candidates_for_positions[position], ballots_for_positions[position], seats
    )
    winner = election.get_winners()
    print(f"=========== {position.upper()} " + "=" * (50 - len(position)))
    print(election)
    elected_gov[position] = winner

In [ ]:
# Show the final elected government in nice Markdown!

from IPython.display import display, Markdown

output = "# Elected BC Gov\n"

for position, winners in elected_gov.items():
    output += f"- **{position}:** " + ", ".join(w.name for w in winners) + "\n"

display(Markdown(output))

